<a href="https://colab.research.google.com/github/meredithoopis/DRL_HF/blob/main/Unit%206/A2C_Advantage_actor_critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [2]:
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 

In [12]:
import os
import gymnasium as gym
import panda_gym
from huggingface_sb3 import load_from_hub, package_to_hub
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from huggingface_hub import notebook_login
import warnings
warnings.filterwarnings('ignore')


In [13]:
#Creating the environment
#Environment ref: https://panda-gym.readthedocs.io/en/latest/usage/environments.html
#In this notebook, we will train the robotic arm to reach, hence the name Panda reach: Dense refers to reward for each step instead of sparse reward, where only when the agent accomplishes the target will it be rewarded

env_id = "PandaReachDense-v3"
env = gym.make(env_id)
s_size = env.observation_space.shape
a_size = env.action_space
print("Observation space", s_size)
print("Sample observation", env.observation_space.sample())

Observation space None
Sample observation OrderedDict([('achieved_goal', array([ 2.5494852,  7.6740546, -1.6337888], dtype=float32)), ('desired_goal', array([-2.1304154, -2.290759 , -8.581627 ], dtype=float32)), ('observation', array([ 4.669556 , -8.370247 ,  3.1603463, -7.136309 ,  5.8120723,
        0.7294128], dtype=float32))])


In [14]:
#Normalize observation and rewards
env = make_vec_env(env_id, n_envs=4)
env = VecNormalize(env, norm_obs = True, norm_reward=True, clip_obs = 10.)

In [15]:
#Training A2C with stable-baselines3
#Ref: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes
model = A2C(policy = "MultiInputPolicy", env = env, verbose = 1)

Using cuda device


In [16]:
model.learn(1_000_000)

Streaming output truncated to the last 5000 lines.
| time/                 |          |
|    fps                | 402      |
|    iterations         | 22300    |
|    time_elapsed       | 1108     |
|    total_timesteps    | 446000   |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.979    |
|    learning_rate      | 0.0007   |
|    n_updates          | 22299    |
|    policy_loss        | 0.0186   |
|    std                | 0.39     |
|    value_loss         | 0.000259 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.96     |
|    ep_rew_mean        | -0.23    |
| time/                 |          |
|    fps                | 402      |
|    iterations         | 22400    |
|    time_elapsed       | 1113     |
|    total_timesteps    | 448000   |
| train/                |          |
|    entropy_loss       | -1.36    |
|    explained_variance 

In [17]:
#Save the model and vecnormalize stats
model.save('A2C-PandaReachDense-v3')
env.save('vec_normalize.pkl')

In [18]:
#Evaluate
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

eval_env.render_mode = "rgb_array"
eval_env.training = False #No update weight at test time
eval_env.norm_reward = False
model = A2C.load('A2C-PandaReachDense-v3')
mean_reward, std_reward = evaluate_policy(model, eval_env)
print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")


Mean reward = -0.22 +/- 0.09


In [19]:
#Push to hub
notebook_login()
!git config --global credential.helper store

In [20]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"lisagrace/a2c-{env_id}", # Change the username
    commit_message="Initial commit",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpll1qb2j6/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpll1qb2j6/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpll1qb2j6/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpll1qb2j6/-step-0-to-step-1000.mp4
ℹ Pushing repo lisagrace/a2c-PandaReachDense-v3 to the Hugging Face
Hub


policy.pth:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/45.2k [00:00<?, ?B/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

a2c-PandaReachDense-v3.zip:   0%|          | 0.00/110k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

vec_normalize.pkl:   0%|          | 0.00/2.65k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/lisagrace/a2c-PandaReachDense-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/lisagrace/a2c-PandaReachDense-v3/commit/4f4fd2d08aa6a4673b08b6f3d39815abc3b831b6', commit_message='Initial commit', commit_description='', oid='4f4fd2d08aa6a4673b08b6f3d39815abc3b831b6', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#Model: https://huggingface.co/lisagrace/a2c-PandaReachDense-v3